# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY') 

Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 19
Python-dotenv could not parse statement starting at line 20
Python-dotenv could not parse statement starting at line 21
Python-dotenv could not parse statement starting at line 22
Python-dotenv could not parse statement starting at line 23
Python-dotenv could not parse statement starting at line 24


### Example 1

#### Create our QandA application

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [5]:
file = 'C:/Users/joshu/lab-langchain-evaluation/data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [7]:
!pip install --upgrade --force-reinstall sentence-transformers

  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached numpy-2.2.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached idna-3.10-py3-n

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.2 which is incompatible.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2025.2.0 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.2 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.2 which is incompatible.
s3fs 2024.9.0 requires fsspec==2024.9.0.*, but you have fsspec 2025.2.0 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.1.0 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26

In [8]:
!pip install docarray

In [9]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})
).from_loaders([loader])

c:\Users\joshu\.anaconda\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [11]:
data[10]

Document(metadata={'source': 'C:/Users/joshu/lab-langchain-evaluation/data/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\r\n\r\nSize & Fit\r\n- Pants are Favorite Fit: Sits lower on the waist.\r\n- Relaxed Fit: Our most generous fit sits farthest from the body.\r\n\r\nFabric & Care\r\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\r\n\r\nAdditional Features\r\n- Relaxed fit top with raglan sleeves and rounded hem.\r\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\r\n\r\nImported.")

In [12]:
data[11]

Document(metadata={'source': 'C:/Users/joshu/lab-langchain-evaluation/data/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

#### Hard-coded examples

In [13]:
from langchain.prompts import PromptTemplate

In [14]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


C:\Users\joshu\AppData\Local\Temp\ipykernel_26720\1856079041.py:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\joshu\AppData\Local\Temp\ipykernel_26720\1856079041.py:56: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"query": query})


answer='Yes, it is available in multiple colors such as grey, black, and blue.'


#### LLM-Generated examples

In [15]:
from langchain.evaluation.qa import QAGenerateChain

In [16]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [17]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [18]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

c:\Users\joshu\.anaconda\Lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [19]:
new_examples[0]

{'qa_pairs': {'query': "What is the recommended sizing advice for the Women's Campside Oxfords mentioned in the document?",
  'answer': "The document advises customers to order their regular shoe size for the Women's Campside Oxfords. If half sizes are not offered, they should order up to the next whole size."}}

In [20]:
data[0]

Document(metadata={'source': 'C:/Users/joshu/lab-langchain-evaluation/data/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.")

In [21]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "What is the recommended sizing advice for the Women's Campside Oxfords mentioned in the document?",
  'answer': "The document advises customers to order their regular shoe size for the Women's Campside Oxfords. If half sizes are not offered, they should order up to the next whole size."},
 {'query': 'What are the dimensions available for the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What features does the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece offer according to the document?",
  'answer': 'The swimsuit features bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom for secure fit and maximum coverage.'},
 {'query': 'What materials is the Refresh Swimwear, V-Neck Tankini Contrasts made from?',
  'answ

#### Combine examples

In [22]:
# examples += new_example
examples += d_flattened

In [23]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'answer': 'Yes'}

In [24]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Based on the provided context, the Cozy Comfort Pullover Set does not have side pockets. It has side seam pockets, a back zip pocket, two elastic mesh water bottle pockets, and a top compartment with a pocket with a double-seal zipper for quick access.'}

### Manual Evaluation - Fun part

In [25]:
import langchain
langchain.debug = True

In [26]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": "Side seam pockets and back zip pocket, with mesh insert for quick drainage.<<<<>>>>>Two elastic mesh water bottle pockets.\r\nTop compartment includes pocket with double-seal zipper for quick access.\r\nSide<<<<>>>>>All pockets have sturdy pocket bags and offer plenty of room for a wallet, cell phone and more.\r\n\r\nGusseted crotch for ease of movement.\r\n\r\nImported.<<<<>>>>>Two elastic mesh water bottle pockets.\r\nTop compartment includes pocket with double-se"
}
[llm/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain > llm

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.'}

In [27]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [28]:
examples += d_flattened

In [29]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What is the recommended sizing advice for the Women's Campside Oxfords mentioned in the document?",
  'answer': "The document advises customers to order their regular shoe size for the Women's Campside Oxfords. If half sizes are not offered, they should order up to the next whole size."},
 {'query': 'What are the dimensions available for the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What features does the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece offer according to the document?",
  'answer': 'The swimsuit features bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabri

In [30]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [31]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "What is the recommended sizing advice for the Women's Campside Oxfords mentioned in the document?",
  'answer': "The document advises customers to order their regular shoe size for the Women's Campside Oxfords. If half sizes are not offered, they should order up to the next whole size.",
  'result': "I don't have information on the Women's Campside Oxfords in the provided context."},
 {'query': 'What are the dimensions available for the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimens

In [32]:
from langchain.evaluation.qa import QAEvalChain

In [33]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [34]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [35]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [36]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: What is the recommended sizing advice for the Women's Campside Oxfords mentioned in the document?
Real Answer: The document advises customers to order their regular shoe size for the Women's Campside Oxfords. If half sizes are not offered, they should order up to the next whole size.
Predicted Answer: I don't have information on the Women's Campside Oxfords in the provided context.

Example 3:
Question: What are the dimensions available for the Recycled Waterhog Dog Mat, Chevron Weave?
Real An

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [39]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = TextLoader("C:/Users/joshu/lab-langchain-evaluation/data/nyc_text.txt")
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})).from_loaders([loader])


llm = ChatOpenAI(temperature= 0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

In [40]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was originally named New Amsterdam by Dutch colonists in 1626. It was later renamed New York when the city came under British control in 1664, after King Charles II of England granted the lands to his brother, the Duke of York. The city was briefly renamed New Orange when the Dutch regained control in 1673, but it has been continuously named New York since November 1674.'

In [41]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was originally named New Amsterdam by Dutch colonists in 1626. It was later renamed New York when the city came under British control in 1664, after King Charles II of England granted the lands to his brother, the Duke of York. The city was briefly renamed New Orange when the Dutch regained control in 1673, but it has been continuously named New York since November 1674.',
 'source_documents': [Document(id='336a6868-c40c-4d54-b4ab-2cf4fd760bb5', metadata={'source': 'C:/Users/joshu/lab-langchain-evaluation/data/nyc_text.txt'}, page_content='The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York City is home to more than 3.2 million residents born outside the U.S., the largest foreign-born population of any city in the world as of 2016.New Yor

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [42]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [43]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain). 
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [44]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'Manhattan (New York County) has the highest population density of any borough in New York City.'

In [63]:

key_mapping = {
    "query": "question",
    "result": "answer",
    "contexts": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [88]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': []}

In [47]:
!pip install --no-cache-dir recordclass

In [48]:
!pip install ragas==0.1.9

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu118 requires torch==2.6.0+cu118, but you have torch 2.6.0 which is incompatible.
torchvision 0.21.0+cu118 requires torch==2.6.0+cu118, but you have torch 2.6.0 which is incompatible.


In [89]:
from ragas.integrations.langchain import EvaluatorChain 
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [90]:
# Recheck the result that we are going to validate.
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': []}

**Faithfulness**

In [91]:
result_updated["contexts"] = [] # Provide an appropriate value


In [92]:
import nest_asyncio
nest_asyncio.apply()


In [93]:
print(result_updated)


{'question': 'Which borough of New York City has the highest population?', 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.', 'contexts': []}


In [95]:
eval_result = faithfulness_chain(result_updated)
print("Eval Result:", eval_result)


Eval Result: {'question': 'Which borough of New York City has the highest population?', 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.', 'contexts': [], 'faithfulness': 0.5}


In [96]:
eval_result = faithfulness_chain(result_updated)
eval_result["faithfulness"]

1.0

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [105]:
fake_result = result.copy()

fake_result["result"] = "we are the champions"  
fake_result["answer"] = fake_result.get("answer", "default answer")  
fake_result["question"] = fake_result.get("question", "default question")  
fake_result["contexts"] = fake_result.get("contexts", ["default context"])
eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness"]

1.0

**Context Relevancy**

In [112]:
eval_result = context_recall_chain(result)  # Run the chain
print(eval_result)  # Print the full response


{'query': 'Which borough of New York City has the highest population?', 'result': 'Manhattan (New York County) has the highest population density of any borough in New York City.', 'source_documents': [Document(id='bb569a0d-3b97-468b-ae25-0d7d02c69dc9', metadata={'source': 'C:/Users/joshu/lab-langchain-evaluation/data/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoe

In [113]:
print("Available keys:", eval_result.keys())




Available keys: dict_keys(['query', 'result', 'source_documents', 'question', 'ground_truth', 'contexts', 'context_recall'])


In [114]:
result["question"] = result.get("question", "default question")
result["ground_truth"] = result.get("ground_truth", "default ground truth")
result["contexts"] = result.get("contexts", ["default context"]) 
eval_result = context_recall_chain(result)
eval_result["context_recall"]

0.0

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [116]:
from langchain.schema import Document
fake_result = result.copy()
fake_result["source_documents"] = [Document(page_content="I love christmas")]
eval_result = context_recall_chain(fake_result)
eval_result["context_recall"]

0.0

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [121]:
formatted_inputs = [
    {
        "question": ex["query"],  # Ensure the key exists
        "answer": pred["result"],  # Use prediction as answer
        "contexts":[doc.page_content for doc in pred.get("source_documents", [])]  # ✅ Extract text from documents
    }
    for ex, pred in zip(examples, predictions)
]

In [123]:

# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# evaluate
print("evaluating...")
r = faithfulness_chain.batch(formatted_inputs)
r

evaluating...


[{'question': 'What is the population of New York City as of 2020?',
  'answer': 'The population of New York City as of 2020 is 8,804,190 residents.',
  'contexts': ["New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropolitan area. The majority of New Y

In [125]:
# evaluate context recall
print("evaluating...")
r = context_recall_chain.evaluate(formatted_inputs)
r

evaluating...


AttributeError: 'EvaluatorChain' object has no attribute 'evaluate'